# Despliegue de Modelo - Evaluación Conjunta

Lo primero que haremos, será el ver que es lo que contiene nuestro dataset

In [1]:
# Importamos nuestras librerías

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def leer_datos(ruta):
    df = pd.read_csv(ruta,sep=',')
    return df

In [3]:
entrenamiento = leer_datos("data_evaluacion.csv")
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48841 entries, 0 to 48840
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   39             48841 non-null  int64 
 1   State-gov      48841 non-null  object
 2   77516          48841 non-null  int64 
 3   Bachelors      48841 non-null  object
 4   13             48841 non-null  int64 
 5   Never-married  48841 non-null  object
 6   Adm-clerical   48841 non-null  object
 7   Not-in-family  48841 non-null  object
 8   White          48841 non-null  object
 9   Male           48841 non-null  object
 10  2174           48841 non-null  int64 
 11  0              48841 non-null  int64 
 12  40             48841 non-null  int64 
 13  United-States  48841 non-null  object
 14  <=50K          48841 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


Como vemos, nuestro dataset consta de 48841 datos, sin embargo, colocaremos encabezados basados en la información de nuestro dataset, para que nos sea más fácil entender las columnas que estamos manejando.

In [4]:
# Cargamos el archivo CSV de la evaluacion
file_path = 'data_evaluacion.csv'
df = pd.read_csv(file_path, header=None)  # header=None indica que el archivo CSV no tiene encabezado

# Asignamos títulos a las columnas
df.columns = ['Edad', 'ClaseObrera', 'PesoFinal', 'Educacion', 'EducacionNum', 'EstadoMarital', 'Ocupacion', 'EstadoCivil', 'Raza', 'Sexo', 'GananciaCapital'
, 'PerdidaCapital', 'HorasPorSemana', 'Pais', 'Salario']

# Guardamos el DataFrame con los nuevos títulos de columna
df.to_csv('data_evaluacion_encabezados.csv', index=False)

print("Archivo CSV con encabezados guardado con éxito.")

Archivo CSV con encabezados guardado con éxito.


In [5]:
# Volvemos a ver si nuestro nuevo archivo tiene los mismos datos

df = leer_datos("data_evaluacion_encabezados.csv")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Edad             48842 non-null  int64 
 1   ClaseObrera      48842 non-null  object
 2   PesoFinal        48842 non-null  int64 
 3   Educacion        48842 non-null  object
 4   EducacionNum     48842 non-null  int64 
 5   EstadoMarital    48842 non-null  object
 6   Ocupacion        48842 non-null  object
 7   EstadoCivil      48842 non-null  object
 8   Raza             48842 non-null  object
 9   Sexo             48842 non-null  object
 10  GananciaCapital  48842 non-null  int64 
 11  PerdidaCapital   48842 non-null  int64 
 12  HorasPorSemana   48842 non-null  int64 
 13  Pais             48842 non-null  object
 14  Salario          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [6]:
# Verificamos si hay valores nulos en las columnas

print(df.isnull().sum())

Edad               0
ClaseObrera        0
PesoFinal          0
Educacion          0
EducacionNum       0
EstadoMarital      0
Ocupacion          0
EstadoCivil        0
Raza               0
Sexo               0
GananciaCapital    0
PerdidaCapital     0
HorasPorSemana     0
Pais               0
Salario            0
dtype: int64


## Limpieza de Datos

In [7]:
# Convertimos variables categóricas a numéricas
le = LabelEncoder()
categorical_columns = ['ClaseObrera', 'Educacion', 'EstadoMarital', 'Ocupacion', 'EstadoCivil', 'Raza', 'Sexo', 'Pais', 'Salario']
for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

# Separamos características y variable objetivo
X = df.drop('Salario', axis=1)
y = df['Salario']

# Dividimos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalamos las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# NORMALIZACION DE CARACTERISTICAS NUMERICAS
scaler = StandardScaler()
numeric_columns = X.select_dtypes(include=[np.number]).columns
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test_scaled[numeric_columns] = scaler.transform(X_test[numeric_columns])


## Entrenamiento del Modelo Regresión Logística

In [8]:
# Función para entrenar y evaluar modelos
def train_evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} - Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))
    return model, accuracy

# Regresión Logística
lr_model, lr_accuracy = train_evaluate_model(LogisticRegression(random_state=42), X_train_scaled, X_test_scaled, y_train, y_test, "Logistic Regression")


Logistic Regression - Accuracy: 0.8237281195618794
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      7414
           1       0.72      0.44      0.55      2355

    accuracy                           0.82      9769
   macro avg       0.78      0.69      0.72      9769
weighted avg       0.81      0.82      0.81      9769



## Identificamos Overfiting o Underfiting

In [9]:
def check_fit(model, X_train, X_test, y_train, y_test, model_name):
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    print(f"{model_name}:")
    print(f"  Train Accuracy: {train_accuracy}")
    print(f"  Test Accuracy: {test_accuracy}")
    if train_accuracy > test_accuracy + 0.05:
        print("  Posible overfitting")
    elif test_accuracy > train_accuracy + 0.05:
        print("  Posible underfitting")
    else:
        print("  Buen ajuste")

# Verificar ajuste para cada modelo
check_fit(lr_model, X_train_scaled, X_test_scaled, y_train, y_test, "Logistic Regression")

Logistic Regression:
  Train Accuracy: 0.8251989865124255
  Test Accuracy: 0.8237281195618794
  Buen ajuste
